In [1]:
%pylab inline

from pycuda.compiler import SourceModule
import theano
import theano.tensor as T
import theano.tensor.signal.conv
import theano.misc.pycuda_init
import theano.sandbox.cuda as cuda

Populating the interactive namespace from numpy and matplotlib


C:\Users\scnerd\Anaconda3\envs\Anaconda34\lib\site-packages\theano\configdefaults.py:1695: UserWarning: Theano does not recognise this flag: Value
  warnings.warn('Theano does not recognise this flag: {0}'.format(key))
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/scnerd/AppData/Local/Theano/compiledir_Windows-10-10.0.14931-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.4.5-64/tmpetptz2j6/m91973e5c136ea49268a916ff971b7377.lib and object C:/Users/scnerd/AppData/Local/Theano/compiledir_Windows-10-10.0.14931-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.4.5-64/tmpetptz2j6/m91973e5c136ea49268a916ff971b7377.exp

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5105)
C:\Users\scnerd\Anaconda3\envs\Anaconda34\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
class PyCUDAHist2D(theano.Op):
    __props__ = ('xres', 'yres', 'xmin', 'xptp', 'ymin', 'yptp', 'length', 'num_blocks', 'threads')

    def __init__(self, width, height, xmin=-1.0, xmax=1.0, ymin=-1.0, ymax=1.0, length=16, num_blocks=4, threads=2**10):
        self.xres = width
        self.yres = height
        self.xmin = xmin
        self.xptp = xmax - xmin
        self.ymin = ymin
        self.yptp = ymax - ymin
        self.length = length
        self.num_blocks = num_blocks
        self.threads = threads

    def make_node(self, x, y, w):
        x = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(x))
        y = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(y))
        w = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(w))
        out_t = cuda.CudaNdarrayType((False, False), dtype='float32')()
        out_x = cuda.CudaNdarrayType((False,))()
        out_y = cuda.CudaNdarrayType((False,))()
        assert x.dtype == 'float32'
        assert y.dtype == 'float32'
        assert w.dtype == 'float32'
        return theano.Apply(self, [x, y, w], [out_t, out_x, out_y])

    # Based on
    #https://devblogs.nvidia.com/parallelforall/gpu-pro-tip-fast-histograms-using-shared-atomics-maxwell/
    def make_thunk(self, node, storage_map, _, _2):

        # Creates a histogram in BLOCKS_X pieces, which then need to be added together
        code = [
        '''
        #define SZ ({xres} * {yres})
        
        __global__ void histogram2d(const float *in_x, const float *in_y, const float *in_w, int n, float *out, float *out_binx, float *out_biny) {{
            int start = threadIdx.x + {length} * blockDim.x * (blockIdx.y * gridDim.x + blockIdx.x);
                
            __shared__ float block_out[SZ];
            for(int i = threadIdx.x; i < SZ; i += blockDim.x) {{
                block_out[i] = (float)0.0;
            }}

            __syncthreads();
            for(int i = 0; i < {length} && start + i * {length} < n; i++) {{
                int pos = start + i * {length};
                float x = in_x[pos];
                float y = in_y[pos];
                int xbin = (int) (((x - {xmin}) / {xptp}) * {xres});
                int ybin = (int) (((y - {ymin}) / {yptp}) * {yres});

                if (0 <= xbin && xbin < {xres} && 0 <= ybin && ybin < {yres}) {{
                    out_binx[pos] = (float)xbin;
                    out_biny[pos] = (float)ybin;
                    atomicAdd(&out[ybin * {xres} + xbin], in_w[pos]);
                }}
            }}
            
            __syncthreads();
            for(int i = threadIdx.x; i < SZ; i += blockDim.x) {{
                atomicAdd(&out[SZ * blockIdx.x + i], block_out[i]);
            }}
        }}
        ''',
        '''
        __global__ void histogram2d(const float *in_x, const float *in_y, const float *in_w, int n, float *out, float *out_binx, float *out_biny) {{
            int start = threadIdx.x + {length} * blockDim.x * (blockIdx.y * gridDim.x + blockIdx.x);

            for(int i = 0; i < {length} && start + i * {length} < n; i++) {{
                int pos = start + i * {length};
                float x = in_x[pos];
                float y = in_y[pos];
                int xbin = (int) (((x - {xmin}) / {xptp}) * {xres});
                int ybin = (int) (((y - {ymin}) / {yptp}) * {yres});

                if (0 <= xbin && xbin < {xres} && 0 <= ybin && ybin < {yres}) {{
                    out_binx[pos] = (float)xbin;
                    out_biny[pos] = (float)ybin;
                    atomicAdd(&out[ybin * {xres} + xbin], in_w[pos]);
                }}
            }}
        }}
        ''']
        code = code[1].format(**{p: eval("self.{}".format(p), {'self': self}) for p in self.__props__})
        mod = SourceModule(code)
        cuda_hist = mod.get_function('histogram2d')

        inputs = [storage_map[v] for v in node.inputs]
        outputs = [storage_map[v] for v in node.outputs]

        _x, _y, _w = inputs
        (_out,_outx,_outy) = outputs

        def run_hist():
            x = _x[0]
            y = _y[0]
            w = _w[0]
            n = x.size
            xres = self.xres
            yres = self.yres
            length = self.length
            threads_per_block = self.threads
            num_blocks = min(self.num_blocks, int(np.ceil(n / threads_per_block / length)))
            num_chunks = int(np.ceil(n / num_blocks / threads_per_block / length))
            print(num_chunks, num_blocks, threads_per_block, length)
            if _out[0] is None or _out[0].shape != (yres, xres):
                _out[0] = cuda.CudaNdarray.zeros((yres, xres))
                _outx[0] = cuda.CudaNdarray.zeros((n,))
                _outy[0] = cuda.CudaNdarray.zeros((n,))
            cuda_hist(x, y, w, numpy.int32(n), _out[0], _outx[0], _outy[0], block=(threads_per_block,1,1), grid=(num_blocks, num_chunks))

        return run_hist#, zeros((num_blocks, yres, xres, num_chans), 'float32')
    
def test():
    n = 100000000
    x = np.random.randn(n)
    y = np.random.randn(n)
    w = np.ones(n)
    plt.figure(figsize=(4,4))
    plt.hist2d(x, y, bins=[np.linspace(-10, 10, 101), np.linspace(-10, 10, 101)])
    plt.show(block=False)

    _x = T.fvector('x')
    _y = T.fvector('y')
    _w = T.fvector('w')
    hister = PyCUDAHist2D(100, 100, xmin=-10, xmax=10, ymin=-10, ymax=10, length=1024)
    f = theano.function([_x, _y, _w], hister(_x, _y, _w), allow_input_downcast=True) 

    from time import time
    start = time()
    hist, x_assgn, y_assgn = f(x, y, w)
    print(time() - start)
    hist = np.array(hist)
    x_assgn = np.cast['int'](np.array(x_assgn))
    y_assgn = np.cast['int'](np.array(y_assgn))
    print(hist.shape)
    plt.figure()
    plt.imshow(hist)
    plt.show(block=False)
    print(list(zip(x[:10], y[:10], x_assgn[:10], y_assgn[:10])))
    print(list(zip(x[-10:], y[-10:], x_assgn[-10:], y_assgn[-10:])))
    print(sum(1 for x, y in zip(x_assgn, y_assgn) if x == y and y == 0))
#test()

In [3]:
class PyCUDASlopeKernel(theano.Op):
    __props__ = ('height', 'width')

    def __init__(self, height, width):
        self.height = height
        self.width = width

    def make_node(self, h):
        h = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(h))
        out_dx = cuda.CudaNdarrayType((False, False), dtype='float32')()
        out_dy = cuda.CudaNdarrayType((False, False), dtype='float32')()
        assert h.dtype == 'float32'
        return theano.Apply(self, [h], [out_dx, out_dy])

    # Based on
    #https://devblogs.nvidia.com/parallelforall/gpu-pro-tip-fast-histograms-using-shared-atomics-maxwell/
    def make_thunk(self, node, storage_map, _, _2):

        # Creates a histogram in BLOCKS_X pieces, which then need to be added together
        code = '''
        __device__ __inline__ float val_at(const float *hist, int x, int y) {{
            x = min(max(x, 0), {width} - 1);
            y = min(max(y, 0), {height} - 1);
            return hist[y * {width} + x];
        }}
        
        __device__ __inline__ float clipper(float mid, const float *hist, int x, int y) {{
            // return min(mid, val_at(hist, x, y));
            return val_at(hist, x, y);
        }}

        __global__ void slope_kernel(const float *histogram, float *dx, float *dy) {{
            int x = (blockIdx.x * blockDim.x + threadIdx.x);
            int y = (blockIdx.y * blockDim.y + threadIdx.y);
            if (x < {width} && y < {height}) {{
                float v4 = val_at(histogram, x, y);
                float d0 = v4 - clipper(v4, histogram, x-1, y-1);
                float d1 = v4 - clipper(v4, histogram, x,   y-1);
                float d2 = v4 - clipper(v4, histogram, x+1, y-1);
                float d3 = v4 - clipper(v4, histogram, x-1, y  );
                float d5 = v4 - clipper(v4, histogram, x+1, y  );
                float d6 = v4 - clipper(v4, histogram, x-1, y+1);
                float d7 = v4 - clipper(v4, histogram, x,   y+1);
                float d8 = v4 - clipper(v4, histogram, x+1, y+1);

                dx[y * {width} + x] = {root2} * (d2 + d8 - d0 - d6) + d5 - d3;
                dy[y * {width} + x] = {root2} * (d6 + d8 - d0 - d2) + d7 - d1;
            }}
        }}
        '''.format(height=self.height, width=self.width, root2=np.sqrt(2))
        mod = SourceModule(code)
        cuda_kern = mod.get_function('slope_kernel')

        inputs = [storage_map[v] for v in node.inputs]
        outputs = [storage_map[v] for v in node.outputs]

        (_h,) = inputs
        (_outx,_outy) = outputs

        threads = 16
        num_blocks_h = int(np.ceil(self.height / threads))
        num_blocks_w = int(np.ceil(self.width / threads))
        def run_kern():
            if _outx[0] is None or _outx[0].shape != (self.height, self.width):
                _outx[0] = cuda.CudaNdarray.zeros((self.height, self.width))
                _outy[0] = cuda.CudaNdarray.zeros((self.height, self.width))
            cuda_kern(_h[0], _outx[0], _outy[0], block=(threads,threads,1), grid=(num_blocks_w,num_blocks_h))

        return run_kern
    

def sloper(hist):
    r2 = np.cast['float32'](np.sqrt(2)/2)
    dx_kern = np.array([[-r2, 0, r2],[-1, 0, 1],[-r2, 0, r2]])
    dy_kern = np.array([[-r2, -1, -r2],[0, 0, 0],[r2, 1, r2]])
    final_kern = np.cast['float32'](np.stack([dx_kern, dy_kern], axis=0))
    res = T.signal.conv.conv2d(hist, final_kern, border_mode='full')
    return res[0, 1:-1, 1:-1], res[1, 1:-1, 1:-1]
    
def test():
    x = np.arange(100)
    y = np.arange(100)
    x, y = np.meshgrid(x, y)
    g = np.exp(-( (x - 50)**2 / 50 + (y - 50)**2 / 50 ))
    plt.figure()
    plt.imshow(g)
    plt.show(block=False)

    _hist = T.fmatrix('hist')
    #sloper = PyCUDASlopeKernel(100, 100)
    f = theano.function([_hist], sloper(_hist), allow_input_downcast=True)

    from time import time
    start = time()
    dx, dy = f(g)
    print(time() - start)
    print(dx.shape)
    print(dy.shape)
    plt.figure()
    plt.imshow(dx)
    plt.show(block=False)
    plt.figure()
    plt.imshow(dy)
    plt.show(block=False)
#test()

In [5]:
cast = np.cast['float32']
    
height = 240
width = 320
n = 10000000
rebound = cast(0.9)
time_step = cast(0.0001)
time_step = theano.shared(time_step)
hist_length = 2**10
gravity = cast((0, 1000))

sizes = np.ones(n)
_reset_posx = lambda: np.clip(np.hstack([np.random.normal(loc=0.1, scale=0.01, size=n/2), np.random.normal(loc=0.7, scale=0.1, size=n/2)]), 0, 1)
_reset_posy = lambda: np.clip(np.random.normal(loc=0.5, scale=0.1, size=n), 0, 1)
_reset_velx = lambda: np.hstack([np.random.normal(loc=-10, scale=0.01, size=n/2), np.random.normal(loc=-10, scale=0.01, size=n/2)])
_reset_vely = lambda: np.random.normal(loc=0, scale=0.01, size=n)
cuda_cast = lambda v: cuda.CudaNdarray(cast(v))

_posx = theano.shared(cuda_cast(_reset_posx()), name='posx')
_posy = theano.shared(cuda_cast(_reset_posy()), name='posy')
_velx = theano.shared(cuda_cast(_reset_velx()), name='velx')
_vely = theano.shared(cuda_cast(_reset_vely()), name='vely')
_size = theano.shared(cuda_cast(sizes), name='sizes')

_mass = _size ** 2 # we're working in 2D, so this is close enough

hister = PyCUDAHist2D(width, height, xmin=0, xmax=1, ymin=0, ymax=1, length=hist_length)
#sloper = PyCUDASlopeKernel(height, width)

hist, x_assgns, y_assgns = hister(_posx, _posy, _size)
x_assgns = T.cast(x_assgns, 'int32')
y_assgns = T.cast(y_assgns, 'int32')

dx, dy = sloper(hist**2)

dx = dx[y_assgns, x_assgns]
dy = dy[y_assgns, x_assgns]
out_velx = _velx + time_step * dx / _mass + time_step * gravity[0]
out_vely = _vely + time_step * dy / _mass + time_step * gravity[1]
out_posx = _posx + time_step * _velx
out_posy = _posy + time_step * _vely

off_left = T.lt(out_posx, 0)
off_right = T.gt(out_posx, 1)
off_top = T.lt(out_posy, 0)
off_bott = T.gt(out_posy, 1)
rebound_x = off_left | off_right
rebound_y = off_top | off_bott

out_velx = T.switch(rebound_x, cast(rebound) * -out_velx, out_velx)
out_vely = T.switch(rebound_y, cast(rebound) * -out_vely, out_vely)
out_posx = T.switch(off_right, cast(1)-(out_posx-cast(1)), T.switch(off_left, -out_posx, out_posx))
out_posy = T.switch(off_bott, cast(1)-(out_posy-cast(1)), T.switch(off_top, -out_posy, out_posy))

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
step = theano.function([], [],
                       updates=[(_posx, out_posx), (_posy, out_posy),
                                (_velx, out_velx), (_vely, out_vely)],
                       allow_input_downcast=True,
                       profile=True)
render = theano.function([], [T.log(1+hist)], allow_input_downcast=True)
reset = theano.function([], [], updates=[
        (_posx, cuda_cast(_reset_posx())),
        (_posy, cuda_cast(_reset_posy())),
        (_velx, cuda_cast(_reset_velx())),
        (_vely, cuda_cast(_reset_vely())),
    ])

theano.printing.debugprint(step)

C:\Users\scnerd\Anaconda3\envs\Anaconda34\lib\site-packages\ipykernel\__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\scnerd\Anaconda3\envs\Anaconda34\lib\site-packages\ipykernel\__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


GpuElemwise{Composite{Switch(i0, (i1 - i2), Switch(i3, (-i2), i2))}}[(0, 2)] [id A] ''   48
 |GpuFromHost [id B] ''   40
 | |Elemwise{Cast{float32}} [id C] ''   29
 |   |Elemwise{gt,no_inplace} [id D] ''   23
 |     |HostFromGpu [id E] ''   15
 |     | |GpuElemwise{Composite{(i0 + (i1 * i2))}}[(0, 0)] [id F] ''   8
 |     |   |posx [id G]
 |     |   |GpuDimShuffle{x} [id H] ''   4
 |     |   | |<CudaNdarrayType(float32, scalar)> [id I]
 |     |   |velx [id J]
 |     |TensorConstant{(1,) of 1} [id K]
 |CudaNdarrayConstant{[ 2.]} [id L]
 |GpuElemwise{Composite{(i0 + (i1 * i2))}}[(0, 0)] [id F] ''   8
 |GpuFromHost [id M] ''   42
   |Elemwise{Cast{float32}} [id N] ''   31
     |Elemwise{lt,no_inplace} [id O] ''   24
       |HostFromGpu [id E] ''   15
       |TensorConstant{(1,) of 0} [id P]
GpuElemwise{Composite{Switch(i0, (i1 * Composite{(i0 + ((i1 * i2) / i3))}(i2, i3, i4, i5)), Composite{(i0 + ((i1 * i2) / i3))}(i2, i3, i4, i5))}}[(0, 2)] [id Q] ''   74
 |GpuFromHost [id R] ''   41
 | 

In [5]:
lump = np.random.choice(np.arange(n), 0.1 * n, replace=False)

reset()

from tqdm import tqdm
print_every_iter = False
renders = []
try:
    for i in tqdm(range(1000)):
        step()
        r, = render()
        renders.append(np.array(r))
finally:
    step.profile.summary()

C:\Users\scnerd\Anaconda3\envs\Anaconda34\lib\site-packages\ipykernel\__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':
  0%|                                                                                                                                                                                                                                             | 0/1000 [00:00<?, ?it/s]

24 4 1024 1024
24 4 1024 1024


  0%|▏                                                                                                                                                                                                                                  | 1/1000 [00:06<1:41:04,  6.07s/it]

24 4 1024 1024
24 4 1024 1024


  0%|▍                                                                                                                                                                                                                                  | 2/1000 [00:12<1:41:33,  6.11s/it]

...

24 4 1024 1024
24 4 1024 1024


  1%|█▎                                                                                                                                                                                                                                 | 6/1000 [00:37<1:42:16,  6.17s/it]

24 4 1024 1024


Function profiling
  Message: <ipython-input-4-581141df6d6d>:61
  Time in 6 calls to Function.__call__: 3.561615e+01s
  Time in Function.fn.__call__: 3.561565e+01s (99.999%)
  Time in thunks: 3.135791e+01s (88.044%)
  Total compile time: 4.003683e+00s
    Number of Apply nodes: 75
    Theano Optimizer time: 5.460050e-01s
       Theano validate time: 1.603484e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 3.413587e+00s
       Import time 3.556485e-01s

Time in all call to theano.grad() 0.000000e+00s
Time since theano import 92.625s
Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  33.3%    33.3%      10.453s       8.71e-01s     Py      12       2   theano.tensor.subtensor.AdvancedSubtensor
  27.5%    60.8%       8.628s       1.80e-01s     C       48       8   theano.sandbox.cuda.basic_ops.GpuFromHost
  24.0%    84.9%       7.540s       5.46e-02s     C      138      23   theano.tensor.elemwise.Elemwise
  10.5%    95.

KeyboardInterrupt: 

In [31]:
renders = [np.array(r[0]).sum(axis=0).sum(axis=-1) for r in __renders]

In [119]:
for i, rendered in enumerate(tqdm(renders)):
    plt.imsave('data/wave_render_5/frame_%d.png' % i, rendered, cmap=plt.cm.inferno)